In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/lang8dataset/whole_data.csv
/kaggle/input/encoderdecodermodel/Encoder_Decoder8epochs.h5


In [2]:
df = pd.read_csv("/kaggle/input/lang8dataset/whole_data.csv", dtype=str)

In [3]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, TimeDistributed, Dense
from tensorflow.keras.models import Model, load_model
from keras.utils.vis_utils import plot_model
import pickle

In [4]:
len(df)

496295

In [5]:
df = df.drop(['correct_input'], axis = 1)
df = df.drop(['correct_output'], axis = 1)

In [6]:
df =df.rename(columns = {'incorrect': 'input', 'correct' : 'target'})

In [7]:
df

,target,input
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang was him ."
2,His Kanji ability is much better than mine .,His Kanji's ability is much better than me .
3,We have known each other for only half a year ...,We have known each other for only half a year ...
4,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .
...,...,...
496290,"I like Thai , because the pronunciation sounds...","I like Thailand language , because that pronou..."
496291,"I ate kaomangai rice with boiled chickin , t...","I ate kaomangai rise with boild chikin , tom..."
496292,I think it is important to like coriander in o...,I think it is important thing to become to lik...
496293,I went to Umeda station for dating yesterday .,"Yesterday , I went to Umeda station to date ."


In [8]:
tkn_all = Tokenizer(filters='', lower=False, split= ' ', oov_token = "<OOV>", char_level = False)
tkn_all.fit_on_texts(df['input'] + df['target'])

In [9]:
start_token = '<sos>'
stop_token = '<eos>'

In [10]:
print(len(tkn_all.word_index))

144142


In [11]:
tkn_all.word_index = {word: index + 1 for index, word in enumerate(tkn_all.word_index)}
tkn_all.word_index[start_token] = 144143
tkn_all.word_index[stop_token] = 144144
tkn_all.index_word[144143] = '<sos>'
tkn_all.index_word[144144] = '<eos>'

In [ ]:
print(tkn_all.word_index)
len(tkn_all.word_index)

In [15]:
import pickle
with open('my_tokenizer.pkl', 'wb') as f:
    pickle.dump(tkn_all, f)

In [12]:
st_token = tkn_all.word_index['<sos>']
ed_token = tkn_all.word_index['<eos>']

In [13]:
def preprocessing_df_corr(df, col, mxlen):
    seq = tkn_all.texts_to_sequences(start_token + " " + df[col]+ " "+ stop_token)
    seq_padded = pad_sequences(seq, maxlen = mxlen, padding = 'post', truncating= 'post')
    return seq_padded

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train, test = train_test_split(df, test_size = 0.2, random_state = 200)

In [16]:
train.shape

(397036, 2)

In [17]:
vocab_size = len(tkn_all.word_index) + 1

In [18]:
inc_train_seq = preprocessing_df_corr(train,'input', 32)
inc_test_seq =preprocessing_df_corr(test,'input', 32)

corr_train_seq = preprocessing_df_corr(train,'target',32)
corr_test_seq = preprocessing_df_corr(test,'target', 32)

In [21]:
encoder_units = 32
decoder_units = 32
embedding_dim = 32

In [22]:
inputs = Input(shape=(inc_train_seq.shape[1],))
embedded = Embedding(vocab_size, embedding_dim, input_length=inc_train_seq.shape[1])(inputs)
encoder = LSTM(encoder_units, return_state=True)
encoder_outputs, state_h, state_c = encoder(embedded)
encoder_states = [state_h, state_c]

decoder = LSTM(decoder_units, return_sequences=True)
decoder_outputs = decoder(embedded, initial_state=encoder_states)

dense = TimeDistributed(Dense(vocab_size, activation='softmax'))
outputs = dense(decoder_outputs)

In [23]:
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [19]:
from tensorflow import keras
model_new = keras.models.load_model('/kaggle/input/encoderdecodermodel/Encoder_Decoder8epochs.h5')

In [20]:
from keras.callbacks import CSVLogger

In [21]:
log_csv = CSVLogger('logs.csv', separator=",", append=False)

In [22]:
import re
def correct_sentence(sentence):
    sequence = tkn_all.texts_to_sequences([start_token + " " + sentence + " " + stop_token])
    padded = pad_sequences(sequence, maxlen=inc_train_seq.shape[1], padding = 'post', truncating = 'post')
    encoded = model.predict(padded)
    y = np.argmax(encoded, axis = 2)
    y = np.reshape(y, (1,32))
    decoded = []
    decoded_np = np.ndarray(32)
    for i in range(encoded.shape[1]):
        word_index = np.argmax(encoded[0, i, :])
        decoded_np = np.append(decoded_np,word_index)
        if word_index > 0:
            decoded.append(tkn_all.index_word[word_index])
            if word_index == tkn_all.word_index[stop_token]:
                break
        else:
            decoded.append('<OOV>')
    return ' '.join(decoded)


def correctAllsentences(input_sentences, target_sentences):
  target_value = target_sentences
  value = []
  for i in input_sentences:
    sen = correct_sentence(i)
    sen = sen.replace("<OOV>" , "")
    sen = sen.replace("<sos>", "")
    sen = sen.replace("<eos>", "")
    sen = re.sub(" +", " " , sen)
    if sen[0] == " ":
      sen = sen[1:]
    value.append(sen)
  return value, target_value


In [ ]:
from nltk.translate.bleu_score import sentence_bleu 
def calculate_bleu(candidate, references):
  bleu_score = []
  for i in range(len(candidate)):
      score = sentence_bleu(references, candidate, weights=(0.5,0.5,0,0))
      bleu_score.append(score)
  return bleu_score

In [ ]:
import csv
class CustomCallback(keras.callbacks.Callback):
  def __init__(self, candidates=[], references=[]):
    self.candidates, self.references = correctAllsentences(df['input'][900:1000], df['target'][900:1000])
    self.bleu_list = []
    self.epoch_list = []
  def on_epoch_end(self, epoch, logs=None):
    keys = list(logs.keys())
    bleu_score = calculate_bleu(self.candidates, self.references)
    print("\nThe bleu Score of epoch :  {} is ".format(epoch), sum(bleu_score)/ len(bleu_score))
    self.bleu_list.append(sum(bleu_score)/ len(bleu_score))
    self.epoch_list.append(epoch)
  def on_train_end(self, logs=None):
    print(self.bleu_list)
    print(self.epoch_list)
    with open('data.csv', 'w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(['Epoch', 'Bleu'])

      for i in range(len(self.epoch_list)):
          writer.writerow([self.epoch_list, self.bleu_list])



In [24]:
# model.summary()
model_new.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 32)       4612640     ['input_1[0][0]']                
                                                                                                  
 lstm (LSTM)                    [(None, 32),         8320        ['embedding[0][0]']              
                                 (None, 32),                                                      
                                 (None, 32)]                                                      
                                                                                              

In [25]:
history = model_new.fit(inc_train_seq, corr_train_seq, epochs=2, batch_size=64, validation_split = 0.2, callbacks = [log_csv])

Epoch 1/2
4963/4963 [==============================] - 2729s 548ms/step - loss: 1.4009 - accuracy: 0.7555 - val_loss: 1.5177 - val_accuracy: 0.7442
Epoch 2/2
4963/4963 [==============================] - 2705s 545ms/step - loss: 1.3787 - accuracy: 0.7575 - val_loss: 1.5101 - val_accuracy: 0.7465


In [27]:
model_new.save('/kaggle/working/model_encoder_decoder_10epochs.h5')

In [41]:
history1 = model.fit(inc_train_seq, corr_train_seq, epochs=3, batch_size=64, validation_split = 0.2, callbacks = [log_csv])

Epoch 1/3
4963/4963 [==============================] - 2705s 545ms/step - loss: 1.4974 - accuracy: 0.7474 - val_loss: 1.5607 - val_accuracy: 0.7420
Epoch 2/3
4963/4963 [==============================] - 2707s 545ms/step - loss: 1.4587 - accuracy: 0.7506 - val_loss: 1.5402 - val_accuracy: 0.7442
Epoch 3/3
4963/4963 [==============================] - 2708s 546ms/step - loss: 1.4272 - accuracy: 0.7532 - val_loss: 1.5274 - val_accuracy: 0.7439


In [42]:
model.save('/kaggle/working/model_8.h5')

In [ ]:
history2 = model.fit(inc_train_seq, corr_train_seq, epochs=10, batch_size=64, validation_split = 0.2, callbacks = [log_csv])

In [39]:
output = correct_sentence("I goes college")

1/1 [==============================] - 0s 20ms/step


In [40]:
output

'<sos> I I college college <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>'

In [ ]:
df['input'][1200]

In [ ]:
model.save('/kaggle/working/model_40.h5')